In [3]:
import numpyro
import numpyro.distributions as dist
import pandas as pd

import jax
import jax.numpy as jnp
from jax import random
import numpy as np
import numpyro
import numpyro.distributions as dist
from numpyro.infer import NUTS, MCMC
from numpyro.infer.reparam import TransformReparam, LocScaleReparam
import numpyro.distributions.transforms as T

In [6]:
def irt_2pl(resp, N, K, beta=1, **hyper_params):
    hyper_a_sd = hyper_params.get('a', 0.35)
    hyper_b_sd = hyper_params.get('b', 1.0)
        
    theta1 = numpyro.sample("theta1", dist.Normal(0, 1).expand([N]).to_event(1))
    theta = numpyro.deterministic("theta", theta1 - jnp.mean(theta1, axis=0, keepdims=True))

    log_a_sigma = numpyro.sample("log_a_sigma", dist.HalfNormal(hyper_a_sd))
    a = numpyro.sample("a",
                       dist.TruncatedNormal(1., log_a_sigma, low=0).expand([K]).to_event(1)
    )

    b_sd = numpyro.sample("b_sd", dist.HalfNormal(hyper_b_sd))
    with numpyro.handlers.reparam(config={"b": LocScaleReparam()}):
        b = numpyro.sample("b", dist.Normal(0., b_sd).expand([K]).to_event(1))
  
    logit = (a * (theta[:, None] - b))
    p = jax.nn.sigmoid(logit)

    with numpyro.handlers.scale(scale=beta):
        with numpyro.plate("person", N):
            numpyro.sample(
                "obs",
                dist.Bernoulli(probs=p).to_event(1),  # K をイベント
                obs=resp
            )

In [7]:
models = ['gpt-5-mini', 'gemini-2.5-flash', 'claude-sonnet-4']
df = pd.read_excel("data/JSNDI_20250906.xlsx")

for model in models:
    df.loc[df[df[model] == 1].index, 'Model'] = model

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.